# Transcript Data Correction

For this code, the data for all of the courses that the students have taken will be reduced down to the highest grades they earned in a course so that the course number is not overcounted.

The focus of this will be in reducing A through G courses down to the two highest grades that the student earned and for BC courses the highest single semester grade that they earned.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Place the destination that is desired for the generated file to output
output_final = "C:\\Users\\derek.castleman\\Desktop\\MonthlyDataPull\\November\\Fixed_Grades.xlsx"

In [ ]:
#Import the historical transcript grades for a student

transcript_grades = pd.read_excel(r"C:\Users\derek.castleman\Desktop\PrintQueryToExcel_20221130_121338_39365f7.xlsx")

#Import BC course list

bc_courses = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Copy of Attachement C_A-GBC.xlsx")

## Reading in Data and Labeling BC

The transcript grades for the students do not label the BC courses for them which will be corrected in this section with a new column for BC added to the transcript data for students.

In [ ]:
transcript_grades

In [ ]:
bc_courses

In [ ]:
# Changing column name to make for a smoother merging
bc_courses = bc_courses.rename(columns={'Course ID':'Course#'})
bc_courses

In [ ]:
# Merge the transcipts and the bc courses
merged = pd.merge(transcript_grades, bc_courses, how='left', on=['Course#', 'Course title'])

merged

In [ ]:
# Renaming course column to make it easier for reading
merged = merged.rename(columns={'Course title1':'Course'})
merged

In [ ]:
# Allows for a chance to download the data to make sure the BC are labelled properly

import base64
from IPython.display import HTML

def create_download_link( df, title = "BC Courses", filename = "BC Courses"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(merged)

## Fixing A through G - Non-BC courses

First the A-G courses that students have taken will be filtered out and then the non-BC courses from this group will be filtered out since these classes are semester only classes. The filtered data will be sorted in order from A to Z. Then the top two rows for any course will be selected getting rid of any extra attempts at the course that the student might have taken.

In [ ]:
# Getting the a through g courses
a_g_courses = merged[merged['CSU Req_x'].notna()]
a_g_courses

In [ ]:
# Removing any bc courses since they are semester courses
minus_bc  = a_g_courses[a_g_courses['Crs Lvl(BC: 24)'] != 24]
minus_bc

In [ ]:
# Checking that no bc remain
minus_bc.info()

In [ ]:
# Sort by student, course and mark then take top two values
grades_sorted = minus_bc.sort_values(['Student ID','Course','Mark']).groupby(['Student ID','Course']).head(2)
grades_sorted

## Fixing BC

Since the BC courses are one semester only, the top grade is all that needs to be considered. First the BC courses are filtered out and sorted in a similar fashon as the A-G, but for these only the top grade will be selected.

In [ ]:
merged

In [ ]:
# Getting BC courses
bc_course = merged.loc[merged['Crs Lvl(BC: 24)'] == 24]
bc_course

In [ ]:
# Getting top BC grade
bc_sorted = bc_course.sort_values(['Student ID','Course','Mark']).groupby(['Student ID','Course']).head(1)
bc_sorted

## Adding BC Back to A-G

Now that the A-G and the BC courses have been fixed the BC courses that are A-G will be filtered out then concatenated to the A-G dataframe so that calculations on courses taken and passed can occur.

In [ ]:
# Selecting A through G of the BC courses
a_g_bc_courses = bc_sorted[bc_sorted['CSU Req_x'].notna()]
a_g_bc_courses

In [ ]:
# Merging bc courses that are A through G with the A through G courses
a_g_final = pd.concat([grades_sorted, a_g_bc_courses])

In [ ]:
# Sorting all values by student ID so that all of a students grades are next to each other
a_g_final = a_g_final.sort_values(['Student ID','Course','Mark'])

In [ ]:
a_g_final

## Calculating A through G Passed

Now that the A-G is fixed and all together once again, courses that have been passed will get a value for one in a newly created column called A-G passed and courses failed will get a value of one in a newly created column called A-G failed.

Students will be grouped by their Student ID number and then have the totals in each one of these columns summed up.

A final column of total A-G will be created by adding up the sums for these two columns.

In [ ]:
# Filtering students with a grade greater than D and giving a point for passing a class
a_g_passed = a_g_final[a_g_final['Mark'] < 'D']
a_g_passed['A-G Passed'] = 1
a_g_passed

In [ ]:
# Filtering students with D and under and giving a point for failed class
a_g_failed = a_g_final[a_g_final['Mark'] >= 'D']
a_g_failed['A-G Failed'] = 1
a_g_failed

In [ ]:
# Summing up classes passed for each student
passed_total = a_g_passed.groupby(by=['Student ID','School','Grade'])['A-G Passed'].sum().reset_index()
passed_total

In [ ]:
# Summing up classes failed for each student
failed_total = a_g_failed.groupby(by=['Student ID','School','Grade'])['A-G Failed'].sum().reset_index()
failed_total

In [ ]:
# Merging passes and failed and assigning zero for any students who may not have data for either
merged_ag = pd.merge(passed_total, failed_total, how='outer', on=['Student ID', 'School', 'Grade'])
merged_ag.update(merged_ag[['A-G Passed','A-G Failed']].fillna(0))
merged_ag

In [ ]:
# Creating a new column for total classes
merged_ag['Total A-G Taken'] = merged_ag['A-G Passed'] + merged_ag['A-G Failed']
merged_ag

## BC Courses Passed and Failed

Completing the exact same method for the last section but now doing it for BC courses to find out how many student have take as well as passed or failed. But this has to be done in two steps:

1) BC classes that are A-G which require a grade above D to pass.

2) BC classes that are not A-G which just require a grade above an F to pass.

### A through G BC courses

In [ ]:
bc_sorted

In [ ]:
bc_ag = bc_sorted[bc_sorted['CSU Req_x'].notna()]
bc_ag

In [ ]:
bc_passed = bc_ag[bc_ag['Mark'] < 'D']
bc_passed['BC Passed'] = 1
bc_passed

In [ ]:
bc_failed = bc_ag[bc_ag['Mark'] >= 'D']
bc_failed['BC Failed'] = 1
bc_failed

In [ ]:
bc_notag = bc_sorted[bc_sorted['CSU Req_x'].isna()]
bc_notag

In [ ]:
bc_passed_notag = bc_notag[bc_notag['Mark'] < 'F']
bc_passed_notag['BC Passed'] = 1
bc_passed_notag

In [ ]:
bc_failed_notag = bc_notag[bc_notag['Mark'] >= 'F']
bc_failed_notag['BC Failed'] = 1
bc_failed_notag

In [ ]:
bc_total_passed = pd.concat([bc_passed, bc_passed_notag])
bc_total_passed

In [ ]:
bc_total_failed = pd.concat([bc_failed, bc_failed_notag])
bc_total_failed

In [ ]:
passed_total_bc = bc_total_passed.groupby(by=['Student ID','School','Grade'])['BC Passed'].sum().reset_index()
passed_total_bc

In [ ]:
failed_total_bc = bc_total_failed.groupby(by=['Student ID','School','Grade'])['BC Failed'].sum().reset_index()
failed_total_bc

In [ ]:
merged_bc = pd.merge(passed_total_bc, failed_total_bc, how='outer', on=['Student ID', 'School', 'Grade'])
merged_bc.update(merged_bc[['BC Passed','BC Failed']].fillna(0))
merged_bc

In [ ]:
merged_bc['Total BC Taken'] = merged_bc['BC Passed'] + merged_bc['BC Failed']
merged_bc

In [ ]:
classes_passed_failed = merged_bc = pd.merge(merged_ag, merged_bc, how='outer', on=['Student ID', 'School', 'Grade'])
classes_passed_failed

## Writing File

The final file will be generated as an excel with three sheets: the master list of courses that students have passed that is not cleaned up, the number of A-G courses passed and failed, and the number of BC courses passed and failed.

In [ ]:
with pd.ExcelWriter('ILPMasterCourses.xlsx') as writer:
    a_g_final.to_excel(writer, sheet_name='A through G Courses')
    bc_sorted.to_excel(writer, sheet_name='BC Courses')
    classes_passed_failed.to_excel(writer, sheet_name='Total Pass and Fails')

In [ ]:
writer = pd.ExcelWriter(output_final)

# Write each dataframe to a different worksheet.
a_g_final.to_excel(writer, sheet_name='A through G Courses')
c_sorted.to_excel(writer, sheet_name='BC Courses'
classes_passed_failed.to_excel(writer, sheet_name='Total Pass and Fails')
writer.save()